## Carbon emissions and Biomass tiles

We need to create new tiles for [Global Forest Watch Climate](http://climate.globalforestwatch.org/map/6/-4.56/-49.92/ALL/grayscale/biomass_loss?begin=2001-01-01&end=2015-01-01&dont_analyze=true), from the [Woods Hold Research Centre above ground biomass](http://whrc.org/publications-data/datasets/national-biomass-and-carbon-dataset/).


### Downscaling problem

We need to downscale a time (integer year) array in conjunction with a quantity (amount of carbon released/ha) array. The downscaling problem is as follows:

<img src="./src/problem_sketch.png" alt="Drawing" style="width: 500px;"/>

Such that, progressivley lower Z levels should contain the SUM of the quantity, and the year that relates to the largest quantity over all contributing mixels.

### Data

We will be using:
* Above Ground Biomass (t/ha)
* Using above ground biomass we convert (\*0.5 \*3.67) to Carbondioxide in biomass (tCO2/ha)
* Hansen Forest Data: year of loss, and tree cover with canopy coverage (%)
* A public-readable, masked Year loss dataset we created (`projects/wri-datalab/HansenBinaryYear_vizzuality`). These data are 0 (no loss) or 255 (loss), with a Sum pyramiding policy (important for QualityMosaic approach). 

We need to create a custom pyramiding policy for downscaling, so the lower-resolution tiles display 1) mean carbon or biomass, and 2) integer years that show the year of most loss within the pixel group of z-1.

### Output

These data will be encoded to a png, webmap. With the band data being:
* 1/R: date of most loss (1 - 15 integer years)
* 2/G: total biomass loss (scaled 0 - 255)
* 3/B: carbon loss (scaled 0 - 255), this is calculated from the total biomass data using some multiplication factors.
* 4/A: uncertainty data (scaled 0 - 255), these will be obtained from the WHRC later.


##### Notes
* Previous approaches to this problem used a quality mosaic. Create an image collection, where there is one image per year, and each image is made of the year_loss masked for the specific year, and reduced with a SUM.
* Dave Thaus original approach: https://code.earthengine.google.com/dc7b57afe93471b2597458ae926024d9


In [1]:
import ee
import ee
import argparse
import re
from IPython.display import Image
ee.Initialize()

### Control variables

In [4]:
crs="EPSG:4326"
scale=27.829872698318393  # native resolution of the data in meters
z_levels={0:156000,       # relates zoom to meters/pixel
          1:78000,
          2:39000,
          3:20000,
          4:10000,
          5:4900,
          6:2400,
          7:1200,
          8:611,
          9:305,
          10:152,
          11:76,
          12:38}
max_pixels=65500
full_intensity=255
canopyThresh = 30
thresholds=[10, 15, 20, 25, 30, 50, 75] # tree canopy thresholds 


### Declare Data 

<table style="width:100%">
  <tr>
    <th style="text-align: left; width:30%">Variable</th>
    <th style="text-align: left">Description</th> 
  </tr>
  <tr>
    <td style="text-align: left">hansen_data</td>
    <td style="text-align: left">Original Hansen data V1.3 from UMD/hansen source</td> 
  </tr>
  <tr>
    <td style="text-align: left">lossByYear</td>
    <td style="text-align: left">Mask of Hansen data (0 and 255), one band per year, Sum pyramiding policy.</td> 
  </tr>
   <tr>
    <td style="text-align: left">loss_yr_where_canopy_threshold_met</td>
    <td style="text-align: left">The year of loss, for locations within a specified forest canopy threshold.</td> 
  </tr>
</table>

In [5]:
# Need to run this procedure for a specific % canopy cover (threshold). Specified in previous cell

# Load in data resources here:
# Get an image of tree loss, at a specific threshold.

hansen_data = ee.Image('UMD/hansen/global_forest_change_2015_v1_3')
tree_cover = hansen_data.select('treecover2000') #tree cover (as a percent - use the threshold to isolcate)
tree_cover = tree_cover.mask(tree_cover)
loss_mask = hansen_data.select('loss')
loss_mask = loss_mask.mask(loss_mask) #loss is a mask 1 for loss, 0 for no loss
loss_by_year = hansen_data.select('lossyear').mask(loss_mask) # masked integer year (-2000): 1 to 15 
canopy_threshold_mask = tree_cover.gte(canopyThresh)
threshold_forest = tree_cover.mask(canopy_threshold_mask) # tree cover of a specified canopyThreshold
loss_yr_where_canopy_threshold_met = loss_by_year.mask(canopy_threshold_mask) # year where specified threshold
loss_yr_where_canopy_threshold_met = loss_yr_where_canopy_threshold_met.mask(loss_yr_where_canopy_threshold_met)

lossByYear = ee.Image('projects/wri-datalab/HansenBinaryYear_vizzuality')
lossByYear = lossByYear.mask(lossByYear);

# loss_yr_where_canopy_threshold_met  : Pixels with the years of interest 

In [6]:
# We can get some confirmation we have what we think we have by returning
# visual data for a small area...
point = ee.Geometry.Point(-53.8330078125,-3.9355003860137967)
region_brazil = point.buffer(50000).bounds().getInfo()['coordinates']

Image(url=loss_yr_where_canopy_threshold_met.getThumbUrl({
    'region':region_brazil,
    'bands':'lossyear',
    'min':1,
    'max':15,
    'palette':'425cf4, 41f45b, f44141'
}))

### Biomass and Carbon data

Unfortunatley these data are in pieces, with some discontinuties, and need to be constructed prior to use.

In [7]:
uncertainty = ee.Image('users/davethau/whrc_carbon_test/uncertainty') # These data are old and should be replaced

In [8]:
neotropic = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Neotropic')
africa = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Africa')
australia = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Australia')
tropasia = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Tropical_Asia')
palearctic = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Palearctic')
nearctic = ee.Image('users/mfarina/Biomass_Data_MapV3/WHRC_Biomass_30m_Nearctic')
          
# Combine the individual areas into a single collection
ic = ee.ImageCollection([africa,australia, nearctic,neotropic, palearctic, tropasia])
im = ic.max() # Now we have single image, but with discontinuties
datamask = hansen_data.select('datamask')
mask = datamask.eq(1)
land = mask # Make a land image out of the mask
landmask = im.mask(land) #  Mask land with itself to mask all the zeros (non-land)
#make another collection from the landmask and the full coverage image
ic_with_mask = ee.ImageCollection([landmask, im])
# Finally convert that into a fully contingous image, with 0s where no data over land
biomass = ic_with_mask.max() # biomass: 'Above Ground Biomass (t/ha)
carbon = biomass.multiply(0.5).multiply(3.67);  #units: tCO2/ha (CO2 stored in above ground biomass)
c_loss_at_canopy_threshold = carbon.mask(canopy_threshold_mask)
c_loss_at_canopy_threshold = c_loss_at_canopy_threshold.mask(loss_yr_where_canopy_threshold_met)

In [9]:
# Show carbon stored in above ground biomass for the same area of Brazil as before,
# masked by loss, for locations within our canopy threshold of interest.
# I.E. All the pixels displayed represent carbon that has been lost (2001-2015) for a
# specified forest canopy threshold.

whrcPALETTE = "75322B,607D34,305B1E,114610"

Image(url=c_loss_at_canopy_threshold.getThumbUrl({
    'region':region_brazil,
    'bands':'b1',
    'min':1,
    'max':900,
    'palette': whrcPALETTE
}))

## Downscaling

Now we have the threshold mask, yearloss, biomass, and carbon data we should be able to proceede.

#### Control variables

In [11]:
def reduce(img_at_z_plus_1, z, reducer, z_max=12):
    """This takes the image fixed to a resolution of z+1, the z level, and a
    specified reducer, e.g. ee.reducer.sum() and returns back a reudced image."""
    if (z==z_max): 
        return img_at_z_plus_1
    else:
        return img_at_z_plus_1.reproject(
                    scale=z_levels[z+1],
                    crs=crs
            ).reduceResolution(
                    reducer=reducer,
                    maxPixels=max_pixels
            ).reproject(
                    scale=z_levels[z],
                    crs=crs
            )

    
def create_rgba_image(r,g,b,a):
    """This takes four single band images, and returns back a multi-band
    image ready to be used as an RGB(A) export. The single band images
    should already be converetd to 0-255 int values."""
    return ee.Image([r,g,b,a])


def export_to_GCS(image, settings={}):
    """Here is where the image will be exported to Google Cloud Storage folder"""
    print(f"   Export to GCS: bucket/folder_{settings['threshold']}_{settings['z']}/<x>/<y>.png\n")
    return

Looks like the limitation of this approach is that it cant run Z-levels in iscolation. It seems to have to step progressivley to lower resolutions.

In [30]:
tmp_biomass = None
tmp_uncertainty = None
tmp_carbon = None
for z in sorted(z_levels.keys(), reverse=True):
    if z == 12:
        print(f"Canopy threshold: {canopyThresh}. At native resolution: Z{z}, {z_levels[z]}m/pix")
        # No need to downsample here, we can just export the data at native resolution
        # Create an image with biomass, carbon, yearloss, and uncertainty in bands (R,G,B,A)
        scaled_c_loss = c_loss_at_canopy_threshold.divide(900).multiply(255).byte() # not sure 900 is max...
        scaled_biomass = biomass.divide(400).multiply(255).byte() # not sure 400 is max...
        tmp_img = create_rgba_image(r=loss_yr_where_canopy_threshold_met.byte(),
                                    g=scaled_c_loss,
                                    b=scaled_biomass,
                                    a=uncertainty.byte())
        # Problem here still: Need to find a way to rename the bands, to 0, 1, 2, 3 
        export_to_GCS(tmp_img,settings={'threshold':canopyThresh, 'z':z})
    else:
        print(f"Canopy threshold: {canopyThresh}. Zoom {z}: {z_levels[z]:,g} m/pix")
        # Downsample data here:
        #
        #  ------------------
        #  Here should go the code for the custom YEAR downsampling ....
        #  ------------------
        #
        
        if tmp_carbon:  # if already looped and tmp_carbon exists, use the previous downscaled data
            tmp_carbon = reduce(tmp_carbon, z, ee.Reducer.sum())
        else:
            tmp_carbon = reduce(c_loss_at_canopy_threshold, z, ee.Reducer.sum())
            
        if tmp_biomass:
            tmp_biomass = reduce(tmp_biomass, z, ee.Reducer.sum())            
        else:
            tmp_biomass = reduce(biomass, z, ee.Reducer.sum())

        if tmp_uncertainty:
            tmp_uncertainty = reduce(tmp_uncertainty, z, ee.Reducer.mean())            
        else:
            tmp_uncertainty = reduce(uncertainty, z, ee.Reducer.mean())
        scaled_c_loss = tmp_carbon.divide(900).multiply(255).byte() # again, not sure 900 is max...
        scaled_biomass = tmp_biomass.divide(400).multiply(255).byte() # again, not sure 400 is max...        
        tmp_img = create_rgba_image(r=None,
                                    g=scaled_c_loss,
                                    b=scaled_biomass,
                                    a=tmp_uncertainty.byte())        
        export_to_GCS(tmp_img,settings={'threshold':canopyThresh, 'z':z})
        
        #     special case of most loss year for loss year
        # Create an image with biomass, carbon, yearloss, and uncertainty
        #     image function: input images to place in bands (R,G,B,A)
        # Export the ONE z level to GCS my_data_<threshold>_<z>/<x>/<y>.png

    

Canopy threshold: 30. At native resolution: Z12, 38m/pix
   Export to GCS: bucket/folder_30_12/<x>/<y>.png

Canopy threshold: 30. Zoom 11: 76 m/pix
   Export to GCS: bucket/folder_30_11/<x>/<y>.png

Canopy threshold: 30. Zoom 10: 152 m/pix
   Export to GCS: bucket/folder_30_10/<x>/<y>.png

Canopy threshold: 30. Zoom 9: 305 m/pix
   Export to GCS: bucket/folder_30_9/<x>/<y>.png

Canopy threshold: 30. Zoom 8: 611 m/pix
   Export to GCS: bucket/folder_30_8/<x>/<y>.png

Canopy threshold: 30. Zoom 7: 1,200 m/pix
   Export to GCS: bucket/folder_30_7/<x>/<y>.png

Canopy threshold: 30. Zoom 6: 2,400 m/pix
   Export to GCS: bucket/folder_30_6/<x>/<y>.png

Canopy threshold: 30. Zoom 5: 4,900 m/pix
   Export to GCS: bucket/folder_30_5/<x>/<y>.png

Canopy threshold: 30. Zoom 4: 10,000 m/pix
   Export to GCS: bucket/folder_30_4/<x>/<y>.png

Canopy threshold: 30. Zoom 3: 20,000 m/pix
   Export to GCS: bucket/folder_30_3/<x>/<y>.png

Canopy threshold: 30. Zoom 2: 39,000 m/pix
   Export to GCS: bucket

In [27]:
# I Extracted info from tmp_image for z12 to test...

tmp_img.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [0.00025, 0.0, -180.0, 0.0, -0.00025, 80.0],
   'data_type': {'max': 255,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [1440000, 560000],
   'id': 'lossyear'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 255,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'b1'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 255,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'b1_1'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.0002777777777996449,
    0.0,
    -117.50013889598083,
    0.0,
    -0.0002777777778031704,
    30.0001388898888],
   'data_type': {'max': 255,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [1035000, 216000],
   'id': 'uncertainty'}],
 'id': 'UMD/hansen/global_forest_change_2015_v1_3',
 'properties':

In [29]:
Image(url=tmp_img.getThumbUrl({
    'region':region_brazil,
    'bands':'b1_1',  #<--- this looks bad still, need to re-name these bands somehow..
    'min':1,
    'max':255,
}))